In [183]:
import os
import pysam



In [184]:
sample = "Burkholderia_ambifaria_100X" #salad_irg+
alignments_path = "../outputs/" + sample + "/metaMDBG/contigs_on_reference.Burkholderia_ambifaria.bam"
contigs_path = "../outputs/" + sample + "/metaMDBG/assembly.fasta"
reference_genome_path = "/groups/genscale/nimauric/soil_references/Burkholderia_ambifaria.fasta"


contig_coverage_threshold = 0

head outputs/Burkholderia_ambifaria_5X/metaMDBG/metaquast/results/combined_reference/contigs_reports/minimap_output/assembly.coords
head outputs/Burkholderia_ambifaria_5X/metaMDBG/contigs_on_reference.Burkholderia_ambifaria.bam

In [185]:
### Get the length of each contig
contig_lengths = {} #name : length
contig_name=""
with open(contigs_path, "r") as contigs_file:
    for line in contigs_file:
        if line.startswith(">"):
            contig_name=line.strip()[1:]
            contig_lengths[contig_name] = 0
        else:
            contig_lengths[contig_name] += len(line.strip())

In [186]:
def calculate_ani(alignment):
    length = alignment.query_alignment_length
    if(length == 0) :
        return 0
    mismatches_and_gaps = alignment.get_tag("NM")

    ani_score = 1 - (mismatches_and_gaps / length)
    ani_score = 1 -alignment.get_tag("de")
    return ani_score


In [187]:
filtered_alignemnts = {}
with pysam.AlignmentFile(alignments_path, "rb") as bam:
    for alignment in bam.fetch() :
        contig_name = alignment.query_name
        if alignment.reference_length >= contig_coverage_threshold * contig_lengths[contig_name] :
            reference_name = alignment.reference_name
            start_pos = alignment.reference_start
            end_pos = alignment.reference_end

            # Skip alignments if its average nucleotide identity is lower than 95%
            if calculate_ani(alignment) < 0.95 : 
                continue
            
            if reference_name in filtered_alignemnts :
                # If the alignment overlaps with existing alignments in the same contig, trim it to remove the overlaps
                for i in range(len(filtered_alignemnts[reference_name])):
                    existing_start = filtered_alignemnts[reference_name][i][0]
                    existing_end = filtered_alignemnts[reference_name][i][1]

                    if(start_pos < existing_start and end_pos > existing_end) : 
                        filtered_alignemnts[reference_name][i] = (0,0)
                    if(start_pos <= existing_end and start_pos >= existing_start) :
                        start_pos = existing_end 
                    if(end_pos >= existing_start and end_pos <= existing_end) : 
                        end_pos = existing_start

                # If not empty, add the new alignement
                if start_pos < end_pos : 
                    filtered_alignemnts[reference_name].append((start_pos, end_pos))

            else:
                filtered_alignemnts[reference_name] = [(start_pos, end_pos)]
            #print(filtered_alignemnts)


aligned_length = {}
for a in filtered_alignemnts.keys() : 
    for tuple in filtered_alignemnts[a] :
        if a not in aligned_length.keys() : 
            aligned_length[a] = 0
        aligned_length[a] += tuple[1] - tuple[0]
    
print(aligned_length)



[W::hts_idx_load3] The index file is older than the data file: ../outputs/Burkholderia_ambifaria_100X/metaMDBG/contigs_on_reference.Burkholderia_ambifaria.bam.bai


{'CP066037.1': 3532803, 'CP066038.1': 2613560, 'CP066040.1': 1265635, 'CP066039.1': 38605}


In [188]:
reference_genome_size = 0
with open(reference_genome_path, "r") as reference_genome_file:
    for line in reference_genome_file:
        if not line.startswith(">"):
            reference_genome_size += len(line.strip())

genome_fraction = sum(aligned_length.values()) / reference_genome_size
print(f"Genome Fraction: {genome_fraction*100:.2f}%")



Genome Fraction: 99.03%
